In [1]:
import sys
sys.path.append('../src')

import os
import requests
import json
import pandas as pd
import betfairlightweight
from betfairlightweight import filters
from datetime import datetime, timedelta
import time

from betfair import Betfair

pd.set_option('display.max_columns', None)

In [2]:
app_key = 'ABGJLOlKaLtTsMIp'

username1 = input("Email:")
password1 = input("Password:")

In [3]:
trading = betfairlightweight.APIClient(
        username1, password1, app_key
    )
trading.login_interactive()

trading.session_token

trading.keep_alive()

<KeepAliveResource>

In [4]:
# Set the start and end time to cover the entire day in UTC
today = datetime.utcnow()
start_time = today.replace(hour=0, minute=0, second=0, microsecond=0)
end_time = today.replace(hour=23, minute=59, second=59, microsecond=999999)

In [5]:
# Define filter for horse racing markets
market_filter = betfairlightweight.filters.market_filter(
    event_type_ids=['7'],
    market_countries=['GB'],
    market_type_codes=['WIN'], 
    market_start_time={'from': start_time.isoformat(),
                       'to' : end_time.isoformat()}
)

# Request market catalogue for all horse races on the date
results = trading.betting.list_market_catalogue(
    filter=market_filter,
    market_projection=["RUNNER_DESCRIPTION", 
            "RUNNER_METADATA", 
            "COMPETITION", 
            "EVENT", 
            "EVENT_TYPE", 
            "MARKET_DESCRIPTION", 
            "MARKET_START_TIME",],  # Include runner metadata
    max_results=1000  # Adjust based on expected number of races
)

In [6]:
for i in results:
        print(
            "{0} {1:d}:{2:02d} {3} ({4})".format(
                i.market_id,
                i.market_start_time.hour, 
                i.market_start_time.minute, 
                i.event.venue,  
                i.description.market_type,
            )
        )


1.232045167 16:20 York (WIN)
1.232045109 16:30 Newton Abbot (WIN)
1.232045742 16:45 Newcastle (WIN)
1.232045751 17:15 Newcastle (WIN)
1.232044988 17:30 Chelmsford City (WIN)
1.232045770 17:45 Newcastle (WIN)
1.232044995 18:00 Chelmsford City (WIN)
1.232045788 18:15 Newcastle (WIN)
1.232045001 18:30 Chelmsford City (WIN)
1.232045795 18:45 Newcastle (WIN)
1.232045007 19:00 Chelmsford City (WIN)
1.232045802 19:15 Newcastle (WIN)
1.232045014 19:30 Chelmsford City (WIN)
1.232045811 19:45 Newcastle (WIN)
1.232045020 20:00 Chelmsford City (WIN)


In [7]:
import pandas as pd

# Initialize an empty list to store market and runner details
data = []

# Iterate over each market in the market catalogue
for market in results:
    market_id = market.market_id
    market_name = market.market_name
    market_start_time = market.market_start_time
    market_type = market.description.market_type

    # Extract the location/venue of the race
    event = market.event
    race_location = event.venue if event else "Unknown Location"
    
    # Iterate over each runner in the market
    for runner in market.runners:
        runner_id = runner.selection_id
        runner_name = runner.runner_name
        
        # Append a dictionary with all relevant data to the list
        data.append({
            'market_id': market_id,
            'market_start_time': market_start_time,
            'venue': race_location,
            'market_type': market_type,
            'selection_id': runner_id,
            'horse_name': runner_name,
        })

# Convert the list of dictionaries to a pandas DataFrame
df1 = pd.DataFrame(data)

# Display the DataFrame
print(df1)


       market_id   market_start_time            venue market_type  \
0    1.232045167 2024-08-22 16:20:00             York         WIN   
1    1.232045167 2024-08-22 16:20:00             York         WIN   
2    1.232045167 2024-08-22 16:20:00             York         WIN   
3    1.232045167 2024-08-22 16:20:00             York         WIN   
4    1.232045167 2024-08-22 16:20:00             York         WIN   
..           ...                 ...              ...         ...   
162  1.232045020 2024-08-22 20:00:00  Chelmsford City         WIN   
163  1.232045020 2024-08-22 20:00:00  Chelmsford City         WIN   
164  1.232045020 2024-08-22 20:00:00  Chelmsford City         WIN   
165  1.232045020 2024-08-22 20:00:00  Chelmsford City         WIN   
166  1.232045020 2024-08-22 20:00:00  Chelmsford City         WIN   

     selection_id         horse_name  
0          159486   Miss Information  
1        56986757       Key To Cotai  
2        62597342      Alfa Kellenic  
3        494118

# I need to get the a list of id's from the market_id column and then create a loop for the list_market_book 

In [8]:
market_id_list = df1['market_id'].drop_duplicates().to_list()
len(market_id_list)

15

In [9]:
# Define a delay between requests
delay = 0.05  # Delay in seconds

# Loop through market IDs and fetch data with a delay
market_books = []
for market_id in market_id_list:
    market_book = trading.betting.list_market_book(
        market_ids=[market_id],
        price_projection=betfairlightweight.filters.price_projection(
            price_data=betfairlightweight.filters.price_data(ex_all_offers=True)
        ),
    )
    market_books.append(market_book)
    
    # Print or process the market_book as needed
    print(market_book)
    
    # Sleep to avoid overloading the API
    time.sleep(delay)

[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]
[<MarketBook>]


In [10]:
# Example: if market_books is a list of lists of MarketBook objects
data = []

# Iterate over each list of MarketBook objects
for book_list in market_books:
    # Iterate over each MarketBook object in the current list
    for market_book in book_list:
        market_id = market_book.market_id
        status = market_book.status
        total_matched = market_book.total_matched

        # Iterate over each runner in the market
        for runner in market_book.runners:
            runner_id = runner.selection_id
            runner_status = runner.status
            last_price_traded = runner.last_price_traded
            runner_total_matched = runner.total_matched
            
            # Append a dictionary with all relevant data to the list
            data.append({
                'market_id': market_id,
                'status': status,
                'total_matched': total_matched,
                'selection_id': runner_id,
                'status': runner_status,
                'morning_price': last_price_traded,
            })



# Convert the list of dictionaries into a DataFrame
df2 = pd.DataFrame(data)

df2 = df2[['market_id', 'selection_id', 'status', 'morning_price', 'total_matched']]
df2

,market_id,selection_id,status,morning_price,total_matched
0,1.232045167,159486,ACTIVE,5.7,365083.72
1,1.232045167,56986757,ACTIVE,7.0,365083.72
2,1.232045167,62597342,ACTIVE,16.5,365083.72
3,1.232045167,49411868,ACTIVE,8.8,365083.72
4,1.232045167,43174762,ACTIVE,10.5,365083.72
...,...,...,...,...,...
162,1.232045020,55953214,ACTIVE,6.0,5002.19
163,1.232045020,62819730,ACTIVE,7.8,5002.19
164,1.232045020,23824676,ACTIVE,10.0,5002.19
165,1.232045020,55540354,ACTIVE,17.5,5002.19


# join df1 and df2

In [13]:
df3 = df1.merge(df2, on = ['market_id', 'selection_id'], how = 'left')

df3['market_id'] = df3['market_id'].astype(str)


current_date = datetime.now().strftime('%Y-%m-%d')
file_name = f"{current_date}_data.csv"


df3.to_csv('../data/daily/' + file_name, index=False)
df3

,market_id,market_start_time,venue,market_type,selection_id,horse_name,status,morning_price,total_matched
0,1.232045167,2024-08-22 16:20:00,York,WIN,159486,Miss Information,ACTIVE,5.7,365083.72
1,1.232045167,2024-08-22 16:20:00,York,WIN,56986757,Key To Cotai,ACTIVE,7.0,365083.72
2,1.232045167,2024-08-22 16:20:00,York,WIN,62597342,Alfa Kellenic,ACTIVE,16.5,365083.72
3,1.232045167,2024-08-22 16:20:00,York,WIN,49411868,Elim,ACTIVE,8.8,365083.72
4,1.232045167,2024-08-22 16:20:00,York,WIN,43174762,Arisaig,ACTIVE,10.5,365083.72
...,...,...,...,...,...,...,...,...,...
162,1.232045020,2024-08-22 20:00:00,Chelmsford City,WIN,55953214,Smooth Silesie,ACTIVE,6.0,5002.19
163,1.232045020,2024-08-22 20:00:00,Chelmsford City,WIN,62819730,Dusk Dame,ACTIVE,7.8,5002.19
164,1.232045020,2024-08-22 20:00:00,Chelmsford City,WIN,23824676,Southbank,ACTIVE,10.0,5002.19
165,1.232045020,2024-08-22 20:00:00,Chelmsford City,WIN,55540354,Lieutenant Rascal,ACTIVE,17.5,5002.19
